<a href="https://colab.research.google.com/github/AkilovM/sentiment_analysis_small_example/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом ноутбуке лежит решение небольшой задачки sentiment-analysis от Яндекса. Ниже текст из задачи.

---

Анализ тональности текста, или сентимент-анализ (от англ. sentiment, «настроение»), выявляет эмоционально окрашенные слова. Этот инструмент помогает компаниям оценивать, например, реакцию на запуск нового продукта в интернете. На разбор тысячи отзывов человек потратит несколько часов, а компьютер — пару минут.

Оценить тональность — значит отметить текст как позитивный или негативный. То есть мы решаем задачу классификации, где целевой признак равен «1» для положительного текста и «0» для отрицательного. Признаки — это слова из корпуса и их величины bag-of-words для каждого текста.

Ваша задача — обучить логистическую регрессию определять тональность твитов.

---

Тренировочные данные по ссылке:

```
https://code.s3.yandex.net/grade/machine/tweets_lemm_train.csv
```

Столбец `text` содержит текст твита, `positive` — целевую метку для классификации, `lemm_text` — лемматизированный текст.

Тестовые данные находятся по ссылке:

```
https://code.s3.yandex.net/grade/machine/tweets_lemm_test.csv
```

Разметку тональности твитов из тестового набора данных можно найти в файле

```
https://storage.yandexcloud.net/mle-data/tweets_lemm_test_labels.csv
```

Столбец `text` содержит текст твита, `positive` — метку тональности твита, `lemm_text` — лемматизированный текст.

---

**Инструкция по выполнению проекта**

Подсчитайте коэффициенты bag-of-words для текстов. Лемматизированные тексты твитов для обучения находятся в файле `tweets_lemm_train.csv`. Целевой признак вы найдёте в столбце `positive`.

Для получения векторов мешка слов используйте CountVectorizer со следующими гиперпараметрами: `min_df=5`, `max_df=0.7`.

Обученной моделью классификации определите результаты предсказания для тестовой выборки твитов, которая лежит в файле `tweets_lemm_test.csv`. В этой выборке целевого признака нет. Сохраните предсказания в столбце `positive`. Таблицу с результатом сохраните как csv-файл, но не указывайте расширение (например, назовите файл “predictions”), чтобы тренажёр принял файл.

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

!wget https://code.s3.yandex.net/grade/machine/tweets_lemm_train.csv
!wget https://code.s3.yandex.net/grade/machine/tweets_lemm_test.csv
!wget https://storage.yandexcloud.net/mle-data/tweets_lemm_test_labels.csv

train = pd.read_csv('tweets_lemm_train.csv')
test = pd.read_csv('tweets_lemm_test.csv')
test_labels = pd.read_csv('tweets_lemm_test_labels.csv')

--2025-08-02 13:57:43--  https://code.s3.yandex.net/grade/machine/tweets_lemm_train.csv
Resolving code.s3.yandex.net (code.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to code.s3.yandex.net (code.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1264499 (1.2M) [text/csv]
Saving to: ‘tweets_lemm_train.csv’

tweets_lemm_train.c 100%[===================>]   1.21M  1.64MB/s    in 0.7s    

2025-08-02 13:57:45 (1.64 MB/s) - ‘tweets_lemm_train.csv’ saved [1264499/1264499]

--2025-08-02 13:57:45--  https://code.s3.yandex.net/grade/machine/tweets_lemm_test.csv
Resolving code.s3.yandex.net (code.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to code.s3.yandex.net (code.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752043 (734K) [text/csv]
Saving to: ‘tweets_lemm_test.csv’

tweets_lemm_test.cs 100%[===================>] 734.42K  1.04MB/s    in 0.7s    

20

In [8]:
train

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...
...,...,...,...
4995,Вроде дружили 10 лет и в один момент все разру...,0,вроде дружить год и в один момент весь разруши...
4996,"@m_gulko \r\nПоэтому и поздравляю заранее, что...",0,поэтому и поздравлять заранее что не получитьс...
4997,"RT @kubudojede: черт, ну зачем они переделали ...",0,черта ну зачем они переделать
4998,"RT @xidewojopiba: Вроде бы и любим друг друга,...",0,вроде бы и любим друг друг и быть вместе не мо...


In [9]:
test

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...
...,...,...
2995,"жуйк, ачивки в контре — зло! мой младший брат ...",жуйк ачивка в контра зло мой младший брат втян...
2996,Не хочу идти на танцы :( http://t.co/5OdPjbYXOC,не хотеть идти на танец
2997,"На улице пусто!ни людей,ни машин,наверно холод...",на улица пусто ни человек ни машина наверно хо...
2998,По-моему мы и ходили за водкой в три часа за э...,по мой мы и ходить за водка в три час за это ч...


In [10]:
test_labels

,text,positive,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,1,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,1,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",1,учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,1,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,1,квн быть отличный оооочень понравиться женский...
...,...,...,...
2995,"жуйк, ачивки в контре — зло! мой младший брат ...",0,жуйк ачивка в контра зло мой младший брат втян...
2996,Не хочу идти на танцы :( http://t.co/5OdPjbYXOC,0,не хотеть идти на танец
2997,"На улице пусто!ни людей,ни машин,наверно холод...",0,на улица пусто ни человек ни машина наверно хо...
2998,По-моему мы и ходили за водкой в три часа за э...,0,по мой мы и ходить за водка в три час за это ч...


In [14]:
count_vect = CountVectorizer(min_df=5, max_df=0.7)
bow_train = count_vect.fit_transform(train['lemm_text'])
bow_test = count_vect.transform(test['lemm_text'])

model = LogisticRegression()
model.fit(bow_train, train['positive'])

prediction = model.predict(bow_test)
accuracy = accuracy_score(test_labels['positive'], prediction)
print(accuracy)

0.6436666666666667
